In [1]:
 import sys,os
 print(sys.executable)
 print(sys.version)
 print(sys.version_info)

/home/antslab/anaconda3/bin/python
3.7.7 (default, Mar 26 2020, 15:48:22) 
[GCC 7.3.0]
sys.version_info(major=3, minor=7, micro=7, releaselevel='final', serial=0)


In [2]:
import findspark
os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3' #'/home/antslab/anaconda3/bin/python'
findspark.init()
import pyspark
# from pyspark import SparkContext
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
from tqdm import tqdm
# import koalas as ks

logFile = "../spark_data/data/Leo_Spark_Home/README.md"  # Should be some file on your system
appName = "Test App"
master = 'local'
# sc = SparkContext(master, appName)
# ks.__version__

In [3]:
# conf = SparkConf().setAppName(appName).setMaster(master).set("spark.network.timeout","800s").set("spark.driver.memory","64g").set("spark.memory.storageFraction","0.3")
conf = SparkConf()
#remote
# conf.setAppName("pcapAnalyzer")#pcap_analyzer pcapAnalyzer
# conf.setMaster("k8s://https://192.168.50.123:6443")
# conf.set("spark.driver.host", "192.168.50.123")
# conf.set("spark.executor.instances","2") #2
# conf.set("spark.executor.cores","6")
# conf.set("spark.driver.memory","30g")
# conf.set("spark.kubernetes.container.image", "smileocean/spark-py:spark_3.0.0_preview2_python")
# conf.set("spark.kubernetes.namespace", "spark-employee") #spark-employee
# conf.set("spark.kubernetes.authenticate.driver.serviceAccountName","spark-employee")#spark-employee
# conf.set("spark.kubernetes.pyspark.pythonVersion","3")
# conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
# conf.set("spark.sql.execution.arrow.enabled", "true")
#local
conf.set("spark.local.dir", "/mnt/ssd240g/data/Leo_Spark_Home/tmp")
conf.set("spark.executor.cores","12")
conf.set("spark.driver.memory","40g")
conf.set("spark.memory.storageFraction","0.2")
conf.set("spark.driver.maxResultSize","0")
conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
conf.set("spark.sql.execution.arrow.enabled", "true")
conf.set("spark.kubernetes.pyspark.pythonVersion","3")
# sc = SparkContext(conf=conf)
# logData = sc.textFile(logFile).cache()
from pyspark.conf import SparkConf
# spark = SparkSession.builder.appName("pcapAnalyzer").config(conf=conf).getOrCreate() #pcap_analyzer pcapAnalyzer

# spark = SparkSession.builder.master("local").appName("dionaea_analyzer").config(conf=conf).getOrCreate()
spark = SparkSession.builder.master("local").appName("Test_spark").config(conf=conf).getOrCreate()
# spark = SparkSession.builder.appName(name="Test_spark").config().getOrCreate()

spark.conf.set("spark.sql.execution.arrow.enabled", True)
# spark
# conf.set("spark.sql.execution.arrow.enabled", True)
import databricks.koalas as ks
ks.options.display.max_rows = 10
ks.set_option('compute.max_rows', None)
ks.set_option('compute.ops_on_diff_frames', True)
ks.set_option('compute.default_index_type', 'distributed')
print(ks.__version__)
spark.version

0.31.0


'3.0.0-preview2'

In [6]:
%%time 
#6min
dir_path = '/home/antslab/spark_data/honeypot_inter/2020_01_10/dionaea/中華電信/'
# sdf = spark.read.json(dir_path+"*.json")
sdf = spark.read.option("multiline","true").json(dir_path+"*.json")
sdf.show()

Py4JJavaError: An error occurred while calling o105.json.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 4 times, most recent failure: Lost task 0.3 in stage 2.0 (TID 179, 10.244.97.134, executor 1): java.io.FileNotFoundException: File file:/home/antslab/spark_data/honeypot_inter/2020_01_10/dionaea/中華電信/part-00082-e62ea158-e6da-4c0a-b162-dc040e8a5ace-c000.json does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:666)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:987)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:656)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:454)
	at org.apache.hadoop.fs.ChecksumFileSystem$ChecksumFSInputChecker.<init>(ChecksumFileSystem.java:146)
	at org.apache.hadoop.fs.ChecksumFileSystem.open(ChecksumFileSystem.java:347)
	at org.apache.hadoop.fs.F

In [6]:
%%time
kdf = sdf.to_koalas()
kdf

CPU times: user 9.81 ms, sys: 3.91 ms, total: 13.7 ms
Wall time: 117 ms


,ip,log_domain,log_level,message_ip,message_noip,port,timestamp
0,None,python module.c:1028,warning,None,/usr/lib/dionaea/python/dionaea/logsql.py:568 ...,NaN,2020-01-10 00:09:47
1,None,python module.c:1029,warning,None,"\t (time.time(), connection_type, con.transpor...",NaN,2020-01-10 00:09:47
2,None,python module.c:1028,warning,None,/usr/lib/dionaea/python/dionaea/logsql.py:621 ...,NaN,2020-01-10 00:09:47
3,None,python module.c:1029,warning,None,"\t attackid = self.connection_insert( icd, 'ac...",NaN,2020-01-10 00:09:47
4,None,python module.c:1028,warning,None,binding.pyx:1184 in dionaea.core.c_python_ihan...,NaN,2020-01-10 00:09:47
5,None,python module.c:1029,warning,None,\t None,NaN,2020-01-10 00:09:47
6,None,python module.c:1003,warning,None,OperationalError at disk I/O error,NaN,2020-01-10 00:09:47
7,None,python module.c:1028,warning,None,/usr/lib/dionaea/python/dionaea/logsql.py:568 ...,NaN,2020-01-10 00:09:47
8,None,python module.c:1029,warning,None,"\t (time.time(), connection_type, con.transpor...",NaN,2020-01-10 00:09:47
9,None,python module.c:1028,warning,None,/usr/lib/dionaea/python/dionaea/logsql.py:621 ...,NaN,2020-01-10 00:09:47


In [16]:
%%time
test_MDB_dir = '../spark_data/intermediate/honeypot/MDB_CNS_2019-12-11/'
sdf2 = spark.read.json(test_MDB_dir+"amun.events.json")
sdf2.show()

+--------------------+-----------+--------------------+----------+--------------------+--------------------+
|                 _id|    channel|               ident|normalized|             payload|           timestamp|
+--------------------+-----------+--------------------+----------+--------------------+--------------------+
|[5df0318090721a14...|amun.events|52da7d68-cba0-11e...|      true|[119.28.222.106, ...|[2019-12-11 08:00...|
|[5df0318090721a14...|amun.events|52da7d68-cba0-11e...|      true|[193.188.23.18, 6...|[2019-12-11 08:00...|
|[5df0318190721a14...|amun.events|52da7d68-cba0-11e...|      true|[181.65.210.130, ...|[2019-12-11 08:00...|
|[5df0318290721a14...|amun.events|52da7d68-cba0-11e...|      true|[47.74.223.87, 56...|[2019-12-11 08:00...|
|[5df0318290721a14...|amun.events|52da7d68-cba0-11e...|      true|[88.206.78.253, 4...|[2019-12-11 08:00...|
|[5df0318390721a14...|amun.events|52da7d68-cba0-11e...|      true|[34.70.182.59, 56...|[2019-12-11 08:00...|
|[5df0318390721a14.

In [18]:
%%time
kdf2 = ks.read_json(test_MDB_dir+"amun.events.json")
kdf2

CPU times: user 15.3 ms, sys: 3.77 ms, total: 19.1 ms
Wall time: 188 ms


,_id,channel,ident,normalized,payload,timestamp
0,"(5df0318090721a14d015ba86,)",amun.events,52da7d68-cba0-11e8-ac82-3ca82a1c2b72,True,"(119.28.222.106, 14972, initial, 203.187.87.51...","(2019-12-11 08:00:00.083000,)"
1,"(5df0318090721a14d015bb03,)",amun.events,52da7d68-cba0-11e8-ac82-3ca82a1c2b72,True,"(193.188.23.18, 64012, initial, 203.187.87.40,...","(2019-12-11 08:00:00.587000,)"
2,"(5df0318190721a14d015bb20,)",amun.events,52da7d68-cba0-11e8-ac82-3ca82a1c2b72,True,"(181.65.210.130, 55732, initial, 203.187.87.26...","(2019-12-11 08:00:01.089000,)"
3,"(5df0318290721a14d015bb28,)",amun.events,52da7d68-cba0-11e8-ac82-3ca82a1c2b72,True,"(47.74.223.87, 56980, initial, 203.187.87.38, 23)","(2019-12-11 08:00:02.280000,)"
4,"(5df0318290721a14d015bb37,)",amun.events,52da7d68-cba0-11e8-ac82-3ca82a1c2b72,True,"(88.206.78.253, 49447, initial, 203.187.87.53,...","(2019-12-11 08:00:02.786000,)"
5,"(5df0318390721a14d015bb4b,)",amun.events,52da7d68-cba0-11e8-ac82-3ca82a1c2b72,True,"(34.70.182.59, 56579, initial, 203.187.87.14, ...","(2019-12-11 08:00:03.287000,)"
6,"(5df0318390721a14d015bb9b,)",amun.events,52da7d68-cba0-11e8-ac82-3ca82a1c2b72,True,"(119.28.222.106, 15606, initial, 203.187.87.51...","(2019-12-11 08:00:03.796000,)"
7,"(5df0318490721a14d015bbb6,)",amun.events,52da7d68-cba0-11e8-ac82-3ca82a1c2b72,True,"(34.66.68.212, 60833, initial, 203.187.87.32, ...","(2019-12-11 08:00:04.298000,)"
8,"(5df0318490721a14d015bbd2,)",amun.events,52da7d68-cba0-11e8-ac82-3ca82a1c2b72,True,"(186.53.131.130, 62111, initial, 203.187.87.58...","(2019-12-11 08:00:04.801000,)"
9,"(5df0318590721a14d015bbde,)",amun.events,52da7d68-cba0-11e8-ac82-3ca82a1c2b72,True,"(34.66.68.212, 60835, initial, 203.187.87.36, ...","(2019-12-11 08:00:05.313000,)"


In [24]:
id_list = kdf2._id.tolist()
id_list = [x[0] for x in id_list]
len(id_list) == len(set(id_list)) #=>_id不重要，因為每個都不一樣
#channel不重要，因為每個都一樣

2000

In [28]:
ident_list = kdf2.ident.tolist()
# id_list = [x[0] for x in id_list]
print(len(ident_list))
print(len(set(ident_list))) #=>ident_list不重要，因為每個都一樣
#normalized不重要，因為每個都一樣

2000
1


* payload 重要

範例順序: attackerIP='193.188.23.18', attackerPort=64012, connectionType='initial', victimIP='203.187.87.40', victimPort=3389


In [22]:
kk = spark.read.text('hdfs://mike-Z370:9000/test/foo.txt')

7=▽  ]11;?"kk" [New File][>c~                                                                               ~                                                                               ~                                                                               ~                                                                               ~                                                                               ~                                                                               ~                                                                               ~                                                                               ~                                                                               ~                                                                               ~                                                                               ~                                                                               ~      

In [9]:
kdf2.iloc[0,3] == True

True

In [6]:
kf2 = sdf2.to_koalas()
df2 = sdf2.toPandas()
kf2.head(20)

,_id,channel,ident,normalized,payload,timestamp
0,"(5df0318090721a14d015ba86,)",amun.events,52da7d68-cba0-11e8-ac82-3ca82a1c2b72,True,"(119.28.222.106, 14972, initial, 203.187.87.51...","(2019-12-11T00:00:00.083Z,)"
1,"(5df0318090721a14d015bb03,)",amun.events,52da7d68-cba0-11e8-ac82-3ca82a1c2b72,True,"(193.188.23.18, 64012, initial, 203.187.87.40,...","(2019-12-11T00:00:00.587Z,)"
2,"(5df0318190721a14d015bb20,)",amun.events,52da7d68-cba0-11e8-ac82-3ca82a1c2b72,True,"(181.65.210.130, 55732, initial, 203.187.87.26...","(2019-12-11T00:00:01.089Z,)"
3,"(5df0318290721a14d015bb28,)",amun.events,52da7d68-cba0-11e8-ac82-3ca82a1c2b72,True,"(47.74.223.87, 56980, initial, 203.187.87.38, 23)","(2019-12-11T00:00:02.280Z,)"
4,"(5df0318290721a14d015bb37,)",amun.events,52da7d68-cba0-11e8-ac82-3ca82a1c2b72,True,"(88.206.78.253, 49447, initial, 203.187.87.53,...","(2019-12-11T00:00:02.786Z,)"
5,"(5df0318390721a14d015bb4b,)",amun.events,52da7d68-cba0-11e8-ac82-3ca82a1c2b72,True,"(34.70.182.59, 56579, initial, 203.187.87.14, ...","(2019-12-11T00:00:03.287Z,)"
6,"(5df0318390721a14d015bb9b,)",amun.events,52da7d68-cba0-11e8-ac82-3ca82a1c2b72,True,"(119.28.222.106, 15606, initial, 203.187.87.51...","(2019-12-11T00:00:03.796Z,)"
7,"(5df0318490721a14d015bbb6,)",amun.events,52da7d68-cba0-11e8-ac82-3ca82a1c2b72,True,"(34.66.68.212, 60833, initial, 203.187.87.32, ...","(2019-12-11T00:00:04.298Z,)"
8,"(5df0318490721a14d015bbd2,)",amun.events,52da7d68-cba0-11e8-ac82-3ca82a1c2b72,True,"(186.53.131.130, 62111, initial, 203.187.87.58...","(2019-12-11T00:00:04.801Z,)"
9,"(5df0318590721a14d015bbde,)",amun.events,52da7d68-cba0-11e8-ac82-3ca82a1c2b72,True,"(34.66.68.212, 60835, initial, 203.187.87.36, ...","(2019-12-11T00:00:05.313Z,)"


In [17]:
sdf2.collect()[0][4]

Row(attackerIP='119.28.222.106', attackerPort=14972, connectionType='initial', victimIP='203.187.87.51', victimPort=80)

In [24]:
df2.iloc[0,5][0]

'2019-12-11T00:00:00.083Z'

In [22]:
kf2.iloc[0,4][0]

'119.28.222.106'

In [31]:
kf2.columns[4]

'payload'

In [56]:
df2['test'] = df2['payload'].map(test_func)

In [57]:
df2[df2['test'] == 22]

,_id,channel,ident,normalized,payload,timestamp,payload2,test


In [63]:
df2['payload2'] = df2['payload'].astype('str')
df2['connectionType' in df2.payload2]

KeyError: False

In [44]:
df2.payload2[0]

"Row(attackerIP='119.28.222.106', attackerPort=14972, connectionType='initial', victimIP='203.187.87.51', victimPort=80)"

In [26]:
sdf2['payload']

Column<b'payload[0]'>

In [55]:
def test_func(s):
    if s[2] != 'initial':
        print(s)
        return(22)

In [59]:
import pickle
ips = pickle.load(open('..//spark_data/intermediate/pcap/cht_0110_top10.pk','rb'))
ips

[Row(ip.src=['210.242.154.92'], count(1)=10421545),
 Row(ip.src=['210.242.196.178'], count(1)=8256328),
 Row(ip.src=['5.188.211.35'], count(1)=6773568),
 Row(ip.src=['210.242.49.2'], count(1)=6134645),
 Row(ip.src=['185.176.27.250'], count(1)=5548999),
 Row(ip.src=['210.109.97.141'], count(1)=5016321),
 Row(ip.src=['210.212.235.51'], count(1)=3655578),
 Row(ip.src=['92.246.76.77'], count(1)=3140992),
 Row(ip.src=['210.14.69.44'], count(1)=2413578),
 Row(ip.src=['81.22.45.190'], count(1)=2375558)]

In [35]:
kdf_pcap = ks.read_json('../data_hdd4t/中華電信/pcap/snort.2020-01-10_dir/snort.log/inbound_1578658563.json')
kdf_pcap

,_index,_score,_source,_type
0,packets-2020-03-26,None,"(([54], [3], [eth:ethertype:ip:tcp], [Jan 10, ...",pcap_file
1,packets-2020-03-26,None,"(([66], [13], [eth:ethertype:ip:tcp], [Jan 10,...",pcap_file
2,packets-2020-03-26,None,"(([66], [14], [eth:ethertype:ip:tcp], [Jan 10,...",pcap_file
3,packets-2020-03-26,None,"(([66], [15], [eth:ethertype:ip:tcp], [Jan 10,...",pcap_file
4,packets-2020-03-26,None,"(([66], [16], [eth:ethertype:ip:tcp], [Jan 10,...",pcap_file
5,packets-2020-03-26,None,"(([66], [17], [eth:ethertype:ip:tcp], [Jan 10,...",pcap_file
6,packets-2020-03-26,None,"(([66], [18], [eth:ethertype:ip:tcp], [Jan 10,...",pcap_file
7,packets-2020-03-26,None,"(([66], [19], [eth:ethertype:ip:tcp], [Jan 10,...",pcap_file
8,packets-2020-03-26,None,"(([66], [20], [eth:ethertype:ip:tcp], [Jan 10,...",pcap_file
9,packets-2020-03-26,None,"(([54], [21], [eth:ethertype:ip:tcp], [Jan 10,...",pcap_file


In [38]:
kdf_pcap.iloc[0,2]

Row(layers=Row(frame.len=['54'], frame.number=['3'], frame.protocols=['eth:ethertype:ip:tcp'], frame.time=['Jan 10, 2020 20:16:03.392395000 CST'], frame.time_epoch=['1578658563.392395000'], ip.dst=['210.242.146.80'], ip.proto=['6'], ip.src=['92.119.160.107'], tcp.dstport=['65323'], tcp.srcport=['46723']))

In [5]:
kdf_pcap.iloc[0,2][0][3][0]
#重要的idx: 0=>frameLen,3=>CSTime,5=>destIP,7=>srcIP,8=>destPort,9=>srcPort

NameError: name 'kdf_pcap' is not defined

# Pcap

In [5]:
date = "09"
# root_dir = "/home/antslab/data_hdd4t/pcap_process/2020_01_"+str(date)+"/中華電信/pcap_2020_01_"+str(date)+"_dir/snort.log/"
# json_files = next(os.walk(root_dir))[2]
# len(json_files)
parquet_file = "../../data_hdd4t/pcap_process/2020_01_"+str(date)+"/中華電信/pcap_2020_01_"+str(date)+"_dir/snort.log/2020_01_"+str(date)+"_country.parquet"
kf = ks.read_parquet(parquet_file)
# df = kf.to_pandas()
kf

,city,domain,frame.len,frame.number,frame.protocols,frame.time,frame.time_epoch,ip.dst,ip.proto,ip.src,ip.ttl,isp,latitude,longitude,network,subdivisions,tcp.dstport,tcp.flags.ack,tcp.flags.fin,tcp.flags.push,tcp.flags.reset,tcp.flags.syn,tcp.flags.urg,tcp.srcport,tcp.window_size_value,traits_network,country
0,None,maxko-hosting.com,[45],[138533],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203422000 CST]",[1578565243.203422000],[210.242.145.4],[17],[45.95.168.116],[46],Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
1,None,maxko-hosting.com,[45],[138534],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203554000 CST]",[1578565243.203554000],[210.242.145.36],[17],[45.95.168.116],[46],Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
2,None,maxko-hosting.com,[45],[138535],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203610000 CST]",[1578565243.203610000],[210.242.145.37],[17],[45.95.168.116],[46],Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
3,None,maxko-hosting.com,[45],[138536],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203639000 CST]",[1578565243.203639000],[210.242.145.66],[17],[45.95.168.116],[46],Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
4,None,maxko-hosting.com,[45],[138537],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203658000 CST]",[1578565243.203658000],[210.242.145.58],[17],[45.95.168.116],[46],Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
5,None,maxko-hosting.com,[73],[138538],[eth:ethertype:ip:icmp:ip:udp:data],"[Jan 9, 2020 18:20:43.203765000 CST]",[1578565243.203765000],"[45.95.168.116, 210.242.145.4]","[1, 17]","[210.242.145.4, 45.95.168.116]","[64, 46]",Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
6,None,maxko-hosting.com,[45],[138539],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203767000 CST]",[1578565243.203767000],[210.242.145.95],[17],[45.95.168.116],[46],Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
7,None,maxko-hosting.com,[45],[138540],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203776000 CST]",[1578565243.203776000],[210.242.145.102],[17],[45.95.168.116],[46],Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
8,None,maxko-hosting.com,[73],[138541],[eth:ethertype:ip:icmp:ip:udp:data],"[Jan 9, 2020 18:20:43.203781000 CST]",[1578565243.203781000],"[45.95.168.116, 210.242.145.36]","[1, 17]","[210.242.145.36, 45.95.168.116]","[64, 46]",Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
9,None,maxko-hosting.com,[73],[138542],[eth:ethertype:ip:icmp:ip:udp:data],"[Jan 9, 2020 18:20:43.203800000 CST]",[1578565243.203800000],"[45.95.168.116, 210.242.145.37]","[1, 17]","[210.242.145.37, 45.95.168.116]","[64, 46]",Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia


,city,domain,frame.len,frame.number,frame.protocols,frame.time,frame.time_epoch,ip.dst,ip.proto,ip.src,ip.ttl,isp,latitude,longitude,network,subdivisions,tcp.dstport,tcp.flags.ack,tcp.flags.fin,tcp.flags.push,tcp.flags.reset,tcp.flags.syn,tcp.flags.urg,tcp.srcport,tcp.window_size_value,traits_network,country
0,None,maxko-hosting.com,[45],[138533],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203422000 CST]",[1578565243.203422000],[210.242.145.4],[17],[45.95.168.116],[46],Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
1,None,maxko-hosting.com,[45],[138534],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203554000 CST]",[1578565243.203554000],[210.242.145.36],[17],[45.95.168.116],[46],Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
2,None,maxko-hosting.com,[45],[138535],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203610000 CST]",[1578565243.203610000],[210.242.145.37],[17],[45.95.168.116],[46],Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
3,None,maxko-hosting.com,[45],[138536],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203639000 CST]",[1578565243.203639000],[210.242.145.66],[17],[45.95.168.116],[46],Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
4,None,maxko-hosting.com,[45],[138537],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203658000 CST]",[1578565243.203658000],[210.242.145.58],[17],[45.95.168.116],[46],Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
5,None,maxko-hosting.com,[73],[138538],[eth:ethertype:ip:icmp:ip:udp:data],"[Jan 9, 2020 18:20:43.203765000 CST]",[1578565243.203765000],"[45.95.168.116, 210.242.145.4]","[1, 17]","[210.242.145.4, 45.95.168.116]","[64, 46]",Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
6,None,maxko-hosting.com,[45],[138539],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203767000 CST]",[1578565243.203767000],[210.242.145.95],[17],[45.95.168.116],[46],Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
7,None,maxko-hosting.com,[45],[138540],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203776000 CST]",[1578565243.203776000],[210.242.145.102],[17],[45.95.168.116],[46],Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
8,None,maxko-hosting.com,[73],[138541],[eth:ethertype:ip:icmp:ip:udp:data],"[Jan 9, 2020 18:20:43.203781000 CST]",[1578565243.203781000],"[45.95.168.116, 210.242.145.36]","[1, 17]","[210.242.145.36, 45.95.168.116]","[64, 46]",Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
9,None,maxko-hosting.com,[73],[138542],[eth:ethertype:ip:icmp:ip:udp:data],"[Jan 9, 2020 18:20:43.203800000 CST]",[1578565243.203800000],"[45.95.168.116, 210.242.145.37]","[1, 17]","[210.242.145.37, 45.95.168.116]","[64, 46]",Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia


In [6]:
kf2 = kf.iloc[0:100000]
kf2

,city,domain,frame.len,frame.number,frame.protocols,frame.time,frame.time_epoch,ip.dst,ip.proto,ip.src,ip.ttl,isp,latitude,longitude,network,subdivisions,tcp.dstport,tcp.flags.ack,tcp.flags.fin,tcp.flags.push,tcp.flags.reset,tcp.flags.syn,tcp.flags.urg,tcp.srcport,tcp.window_size_value,traits_network,country
0,None,maxko-hosting.com,[45],[138533],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203422000 CST]",[1578565243.203422000],[210.242.145.4],[17],[45.95.168.116],[46],Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
1,None,maxko-hosting.com,[45],[138534],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203554000 CST]",[1578565243.203554000],[210.242.145.36],[17],[45.95.168.116],[46],Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
2,None,maxko-hosting.com,[45],[138535],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203610000 CST]",[1578565243.203610000],[210.242.145.37],[17],[45.95.168.116],[46],Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
3,None,maxko-hosting.com,[45],[138536],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203639000 CST]",[1578565243.203639000],[210.242.145.66],[17],[45.95.168.116],[46],Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
4,None,maxko-hosting.com,[45],[138537],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203658000 CST]",[1578565243.203658000],[210.242.145.58],[17],[45.95.168.116],[46],Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
5,None,maxko-hosting.com,[73],[138538],[eth:ethertype:ip:icmp:ip:udp:data],"[Jan 9, 2020 18:20:43.203765000 CST]",[1578565243.203765000],"[45.95.168.116, 210.242.145.4]","[1, 17]","[210.242.145.4, 45.95.168.116]","[64, 46]",Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
6,None,maxko-hosting.com,[45],[138539],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203767000 CST]",[1578565243.203767000],[210.242.145.95],[17],[45.95.168.116],[46],Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
7,None,maxko-hosting.com,[45],[138540],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203776000 CST]",[1578565243.203776000],[210.242.145.102],[17],[45.95.168.116],[46],Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
8,None,maxko-hosting.com,[73],[138541],[eth:ethertype:ip:icmp:ip:udp:data],"[Jan 9, 2020 18:20:43.203781000 CST]",[1578565243.203781000],"[45.95.168.116, 210.242.145.36]","[1, 17]","[210.242.145.36, 45.95.168.116]","[64, 46]",Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
9,None,maxko-hosting.com,[73],[138542],[eth:ethertype:ip:icmp:ip:udp:data],"[Jan 9, 2020 18:20:43.203800000 CST]",[1578565243.203800000],"[45.95.168.116, 210.242.145.37]","[1, 17]","[210.242.145.37, 45.95.168.116]","[64, 46]",Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia


In [7]:
df = kf2.to_pandas()
df

,city,domain,frame.len,frame.number,frame.protocols,frame.time,frame.time_epoch,ip.dst,ip.proto,ip.src,...,tcp.flags.ack,tcp.flags.fin,tcp.flags.push,tcp.flags.reset,tcp.flags.syn,tcp.flags.urg,tcp.srcport,tcp.window_size_value,traits_network,country
0,None,maxko-hosting.com,[45],[138533],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203422000 CST]",[1578565243.203422000],[210.242.145.4],[17],[45.95.168.116],...,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
1,None,maxko-hosting.com,[45],[138534],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203554000 CST]",[1578565243.203554000],[210.242.145.36],[17],[45.95.168.116],...,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
2,None,maxko-hosting.com,[45],[138535],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203610000 CST]",[1578565243.203610000],[210.242.145.37],[17],[45.95.168.116],...,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
3,None,maxko-hosting.com,[45],[138536],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203639000 CST]",[1578565243.203639000],[210.242.145.66],[17],[45.95.168.116],...,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
4,None,maxko-hosting.com,[45],[138537],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203658000 CST]",[1578565243.203658000],[210.242.145.58],[17],[45.95.168.116],...,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214748375605,Tbilisi,magticom.ge,[66],[634758],[eth:ethertype:ip:tcp],"[Jan 9, 2020 10:03:13.909999000 CST]",[1578535393.909999000],[93.177.147.200],[6],[210.242.148.29],...,[1],[0],[0],[0],[1],[0],[3389],[29200],93.177.144.0/22,Georgia
214748375606,Tbilisi,magticom.ge,[66],[634759],[eth:ethertype:ip:tcp],"[Jan 9, 2020 10:03:13.910020000 CST]",[1578535393.910020000],[93.177.147.200],[6],[210.242.145.13],...,[1],[0],[0],[0],[1],[0],[3389],[29200],93.177.144.0/22,Georgia
214748375607,Tbilisi,magticom.ge,[66],[634761],[eth:ethertype:ip:tcp],"[Jan 9, 2020 10:03:13.910031000 CST]",[1578535393.910031000],[93.177.147.200],[6],[210.242.148.62],...,[1],[0],[0],[0],[1],[0],[3389],[29200],93.177.144.0/22,Georgia
214748375608,Tbilisi,magticom.ge,[66],[634762],[eth:ethertype:ip:tcp],"[Jan 9, 2020 10:03:13.910041000 CST]",[1578535393.910041000],[93.177.147.200],[6],[210.242.148.31],...,[1],[0],[0],[0],[1],[0],[3389],[29200],93.177.144.0/22,Georgia


In [8]:
import pickle
pickle.dump(obj=df,file=open('../spark_data/data/Leo_Spark_Home/tmp/part_df.pkl','wb'))

In [5]:
def framelen_pre(s):
    return int(s[0])
def protocol_pre(s):
    ss = s[0].split(':')
    ss = ss[3:]
    ss = ":".join(ss)
    return ss

In [4]:
sdf = spark.read.parquet("../data_hdd4t/pcap_process/2020_01_09/中華電信/pcap_2020_01_09_dir/snort.log/2020_01_09_country.parquet")
kf = sdf.to_koalas()
print(len(kf))
kf.head(10)

209015830


,city,domain,frame.len,frame.number,frame.protocols,frame.time,frame.time_epoch,ip.dst,ip.proto,ip.src,ip.ttl,isp,latitude,longitude,network,subdivisions,tcp.dstport,tcp.flags.ack,tcp.flags.fin,tcp.flags.push,tcp.flags.reset,tcp.flags.syn,tcp.flags.urg,tcp.srcport,tcp.window_size_value,traits_network,country
0,None,maxko-hosting.com,[45],[138533],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203422000 CST]",[1578565243.203422000],[210.242.145.4],[17],[45.95.168.116],[46],Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
1,None,maxko-hosting.com,[45],[138534],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203554000 CST]",[1578565243.203554000],[210.242.145.36],[17],[45.95.168.116],[46],Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
2,None,maxko-hosting.com,[45],[138535],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203610000 CST]",[1578565243.203610000],[210.242.145.37],[17],[45.95.168.116],[46],Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
3,None,maxko-hosting.com,[45],[138536],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203639000 CST]",[1578565243.203639000],[210.242.145.66],[17],[45.95.168.116],[46],Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
4,None,maxko-hosting.com,[45],[138537],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203658000 CST]",[1578565243.203658000],[210.242.145.58],[17],[45.95.168.116],[46],Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
5,None,maxko-hosting.com,[73],[138538],[eth:ethertype:ip:icmp:ip:udp:data],"[Jan 9, 2020 18:20:43.203765000 CST]",[1578565243.203765000],"[45.95.168.116, 210.242.145.4]","[1, 17]","[210.242.145.4, 45.95.168.116]","[64, 46]",Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
6,None,maxko-hosting.com,[45],[138539],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203767000 CST]",[1578565243.203767000],[210.242.145.95],[17],[45.95.168.116],[46],Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
7,None,maxko-hosting.com,[45],[138540],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203776000 CST]",[1578565243.203776000],[210.242.145.102],[17],[45.95.168.116],[46],Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
8,None,maxko-hosting.com,[73],[138541],[eth:ethertype:ip:icmp:ip:udp:data],"[Jan 9, 2020 18:20:43.203781000 CST]",[1578565243.203781000],"[45.95.168.116, 210.242.145.36]","[1, 17]","[210.242.145.36, 45.95.168.116]","[64, 46]",Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
9,None,maxko-hosting.com,[73],[138542],[eth:ethertype:ip:icmp:ip:udp:data],"[Jan 9, 2020 18:20:43.203800000 CST]",[1578565243.203800000],"[45.95.168.116, 210.242.145.37]","[1, 17]","[210.242.145.37, 45.95.168.116]","[64, 46]",Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia


In [5]:
kf2 = ks.read_parquet("../data_hdd4t/pcap_process/2020_01_09/中華電信/pcap_2020_01_09_dir/snort.log/2020_01_09_country.parquet")
kf2

,city,domain,frame.len,frame.number,frame.protocols,frame.time,frame.time_epoch,ip.dst,ip.proto,ip.src,ip.ttl,isp,latitude,longitude,network,subdivisions,tcp.dstport,tcp.flags.ack,tcp.flags.fin,tcp.flags.push,tcp.flags.reset,tcp.flags.syn,tcp.flags.urg,tcp.srcport,tcp.window_size_value,traits_network,country
0,None,maxko-hosting.com,[45],[138533],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203422000 CST]",[1578565243.203422000],[210.242.145.4],[17],[45.95.168.116],[46],Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
1,None,maxko-hosting.com,[45],[138534],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203554000 CST]",[1578565243.203554000],[210.242.145.36],[17],[45.95.168.116],[46],Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
2,None,maxko-hosting.com,[45],[138535],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203610000 CST]",[1578565243.203610000],[210.242.145.37],[17],[45.95.168.116],[46],Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
3,None,maxko-hosting.com,[45],[138536],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203639000 CST]",[1578565243.203639000],[210.242.145.66],[17],[45.95.168.116],[46],Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
4,None,maxko-hosting.com,[45],[138537],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203658000 CST]",[1578565243.203658000],[210.242.145.58],[17],[45.95.168.116],[46],Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
5,None,maxko-hosting.com,[73],[138538],[eth:ethertype:ip:icmp:ip:udp:data],"[Jan 9, 2020 18:20:43.203765000 CST]",[1578565243.203765000],"[45.95.168.116, 210.242.145.4]","[1, 17]","[210.242.145.4, 45.95.168.116]","[64, 46]",Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
6,None,maxko-hosting.com,[45],[138539],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203767000 CST]",[1578565243.203767000],[210.242.145.95],[17],[45.95.168.116],[46],Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
7,None,maxko-hosting.com,[45],[138540],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203776000 CST]",[1578565243.203776000],[210.242.145.102],[17],[45.95.168.116],[46],Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
8,None,maxko-hosting.com,[73],[138541],[eth:ethertype:ip:icmp:ip:udp:data],"[Jan 9, 2020 18:20:43.203781000 CST]",[1578565243.203781000],"[45.95.168.116, 210.242.145.36]","[1, 17]","[210.242.145.36, 45.95.168.116]","[64, 46]",Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
9,None,maxko-hosting.com,[73],[138542],[eth:ethertype:ip:icmp:ip:udp:data],"[Jan 9, 2020 18:20:43.203800000 CST]",[1578565243.203800000],"[45.95.168.116, 210.242.145.37]","[1, 17]","[210.242.145.37, 45.95.168.116]","[64, 46]",Giganet Internet Szolgaltato Kft,45.1667,15.5,45.95.168.0/22,None,None,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia


In [ ]:
kf2.iloc[0]

In [ ]:
df = sdf.toPandas()
df

In [7]:
for jsonfile in (json_files):
    json_path = root_dir + jsonfile
    sdf = spark.read.option("multiline","true").json(json_path)
    sdf = sdf.select("_source.layers.*")
    kf = sdf.to_koalas()
    kf['frame.len'] = kf['frame.len'].map(framelen_pre)
    kf['frame.protocols'] = kf['frame.protocols'].map(protocol_pre)
    break
# sdf.show()
df = kf.to_pandas()
kf.head(20)

,city,country,domain,frame.len,frame.number,frame.protocols,frame.time,frame.time_epoch,ip.dst,ip.proto,ip.src,ip.ttl,isp,latitude,longitude,network,subdivisions,tcp.dstport,tcp.flags.ack,tcp.flags.fin,tcp.flags.push,tcp.flags.reset,tcp.flags.syn,tcp.flags.urg,tcp.srcport,tcp.window_size_value,traits_network
0,None,United States,googlebot.com,66,[1],tcp,"[Jan 10, 2020 13:00:26.562011000 CST]",[1578632426.562011000],[210.242.148.149],[6],[66.249.79.206],[105],Googlebot,37.751,-97.822,66.249.79.0/24,None,[80],[1],[0],[0],[0],[0],[0],[49532],[12294],66.249.79.128/25
1,None,United States,googlebot.com,66,[2],tcp,"[Jan 10, 2020 13:00:26.562014000 CST]",[1578632426.562014000],[210.242.148.149],[6],[66.249.79.206],[105],Googlebot,37.751,-97.822,66.249.79.0/24,None,[80],[1],[0],[0],[0],[0],[0],[49532],[12305],66.249.79.128/25
2,None,United States,googlebot.com,66,[3],tcp,"[Jan 10, 2020 13:00:26.562018000 CST]",[1578632426.562018000],[210.242.148.149],[6],[66.249.79.206],[105],Googlebot,37.751,-97.822,66.249.79.0/24,None,[80],[1],[0],[0],[0],[0],[0],[49532],[12316],66.249.79.128/25
3,None,United States,googlebot.com,66,[4],tcp,"[Jan 10, 2020 13:00:26.562021000 CST]",[1578632426.562021000],[210.242.148.149],[6],[66.249.79.206],[105],Googlebot,37.751,-97.822,66.249.79.0/24,None,[80],[1],[0],[0],[0],[0],[0],[49532],[12327],66.249.79.128/25
4,None,United States,googlebot.com,66,[5],tcp,"[Jan 10, 2020 13:00:26.562024000 CST]",[1578632426.562024000],[210.242.148.149],[6],[66.249.79.206],[105],Googlebot,37.751,-97.822,66.249.79.0/24,None,[80],[1],[0],[0],[0],[0],[0],[49532],[12338],66.249.79.128/25
5,None,United States,googlebot.com,66,[6],tcp,"[Jan 10, 2020 13:00:26.562037000 CST]",[1578632426.562037000],[210.242.148.149],[6],[66.249.79.206],[105],Googlebot,37.751,-97.822,66.249.79.0/24,None,[80],[1],[0],[0],[0],[0],[0],[49532],[12349],66.249.79.128/25
6,None,United States,googlebot.com,66,[7],tcp,"[Jan 10, 2020 13:00:26.562041000 CST]",[1578632426.562041000],[210.242.148.149],[6],[66.249.79.206],[105],Googlebot,37.751,-97.822,66.249.79.0/24,None,[80],[1],[0],[0],[0],[0],[0],[49532],[12360],66.249.79.128/25
7,None,United States,googlebot.com,66,[8],tcp,"[Jan 10, 2020 13:00:26.562044000 CST]",[1578632426.562044000],[210.242.148.149],[6],[66.249.79.206],[105],Googlebot,37.751,-97.822,66.249.79.0/24,None,[80],[1],[0],[0],[0],[0],[0],[49532],[12371],66.249.79.128/25
8,None,United States,googlebot.com,66,[9],tcp,"[Jan 10, 2020 13:00:26.562048000 CST]",[1578632426.562048000],[210.242.148.149],[6],[66.249.79.206],[105],Googlebot,37.751,-97.822,66.249.79.0/24,None,[80],[1],[0],[0],[0],[0],[0],[49532],[12382],66.249.79.128/25
9,None,United States,googlebot.com,66,[10],tcp,"[Jan 10, 2020 13:00:26.562051000 CST]",[1578632426.562051000],[210.242.148.149],[6],[66.249.79.206],[105],Googlebot,37.751,-97.822,66.249.79.0/24,None,[80],[1],[0],[0],[0],[0],[0],[49532],[12394],66.249.79.128/25


In [16]:
df['frame.protocols'] = df['frame.protocols'].map(protocol_pre)
df

,city,country,domain,frame.len,frame.number,frame.protocols,frame.time,frame.time_epoch,ip.dst,ip.proto,...,tcp.dstport,tcp.flags.ack,tcp.flags.fin,tcp.flags.push,tcp.flags.reset,tcp.flags.syn,tcp.flags.urg,tcp.srcport,tcp.window_size_value,traits_network
0,None,United States,googlebot.com,7156,[1],tcp:http:data,"[Jan 9, 2020 21:28:52.755474000 CST]",[1578576532.755474000],[66.249.71.152],[6],...,[37409],[1],[0],[0],[0],[0],[0],[80],[235],66.249.70.0/23
1,None,United States,googlebot.com,7156,[2],tcp:http:data,"[Jan 9, 2020 21:28:52.755534000 CST]",[1578576532.755534000],[66.249.71.152],[6],...,[37409],[1],[0],[0],[0],[0],[0],[80],[235],66.249.70.0/23
2,None,United States,googlebot.com,7156,[3],tcp:http:data,"[Jan 9, 2020 21:28:52.755596000 CST]",[1578576532.755596000],[66.249.71.152],[6],...,[37409],[1],[0],[0],[0],[0],[0],[80],[235],66.249.70.0/23
3,None,United States,googlebot.com,7156,[4],tcp:http:data,"[Jan 9, 2020 21:28:52.755655000 CST]",[1578576532.755655000],[66.249.71.152],[6],...,[37409],[1],[0],[0],[0],[0],[0],[80],[235],66.249.70.0/23
4,None,United States,googlebot.com,7156,[5],tcp:http:data,"[Jan 9, 2020 21:28:52.755715000 CST]",[1578576532.755715000],[66.249.71.152],[6],...,[37409],[1],[0],[0],[0],[0],[0],[80],[235],66.249.70.0/23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350072,None,United States,googlebot.com,7156,[350163],tcp:http:data,"[Jan 9, 2020 21:30:25.866797000 CST]",[1578576625.866797000],[66.249.71.63],[6],...,[57288],[1],[0],[0],[0],[0],[0],[80],[235],66.249.70.0/23
350073,None,United States,googlebot.com,7156,[350164],tcp:http:data,"[Jan 9, 2020 21:30:25.866858000 CST]",[1578576625.866858000],[66.249.71.63],[6],...,[57288],[1],[0],[0],[0],[0],[0],[80],[235],66.249.70.0/23
350074,None,United States,googlebot.com,7156,[350165],tcp:http:data,"[Jan 9, 2020 21:30:25.866918000 CST]",[1578576625.866918000],[66.249.71.63],[6],...,[57288],[1],[0],[0],[0],[0],[0],[80],[235],66.249.70.0/23
350075,None,United States,googlebot.com,7156,[350166],tcp:http:data,"[Jan 9, 2020 21:30:25.866979000 CST]",[1578576625.866979000],[66.249.71.63],[6],...,[57288],[1],[0],[0],[0],[0],[0],[80],[235],66.249.70.0/23


In [9]:
domain_li = df.domain.tolist()
domains = set(domain_li)
domains

{'111-tataidc.co.in',
 '114-tataidc.co.in',
 '123-tataidc.co.in',
 '163data.com.cn',
 '182-airtel.com',
 '1toc.com.br',
 '2com.net',
 '3bb.co.th',
 None,
 'a3fiber.se',
 'abinee.org.br',
 'actcorp.in',
 'adinet.com.uy',
 'adnsl.com',
 'adsl-pool.sx.cn',
 'adslplus.ch',
 'airtel.in',
 'airtelbroadband.in',
 'algarnetsuper.com.br',
 'algartelecom.com.br',
 'algx.net',
 'altitud.net',
 'amazonaws.com',
 'arbor-observatory.com',
 'aruba.it',
 'arubacloud.com',
 'as13285.net',
 'as8758.net',
 'as9105.com',
 'asianet.co.th',
 'astranet.ru',
 'ats99.ru',
 'aueb.gr',
 'avangarddsl.ru',
 'axnet.com.br',
 'axntechnologies.in',
 'axtel.net',
 'azeronline.com',
 'baidu.jp',
 'bashtel.ru',
 'bbox.fr',
 'bbtec.net',
 'belgacom.be',
 'bell.ca',
 'bezeqint.net',
 'bredband2.com',
 'brphonia.com.br',
 'btopenworld.com',
 'bulsat.com',
 'cablecolor.hn',
 'cantv.net',
 'catch.no',
 'cdn77.com',
 'cea.ru',
 'cednet.com.br',
 'censys.io',
 'cgocable.ca',
 'cgocable.net',
 'chinamobile.com',
 'cipherwave.ne

In [10]:
df.columns

Index(['city', 'country', 'domain', 'frame.len', 'frame.number',
       'frame.protocols', 'frame.time', 'frame.time_epoch', 'ip.dst',
       'ip.proto', 'ip.src', 'ip.ttl', 'isp', 'latitude', 'longitude',
       'network', 'subdivisions', 'tcp.dstport', 'tcp.flags.ack',
       'tcp.flags.fin', 'tcp.flags.push', 'tcp.flags.reset', 'tcp.flags.syn',
       'tcp.flags.urg', 'tcp.srcport', 'tcp.window_size_value',
       'traits_network'],
      dtype='object')

In [12]:
protocols_li = df['frame.protocols'].tolist()
protocols_li = [x[0] for x in protocols_li]
pro_se = set(protocols_li)
pro_se # 從第三格開始embed成?bytes

{'eth:ethertype:ip:icmp',
 'eth:ethertype:ip:icmp:data',
 'eth:ethertype:ip:icmp:ip:ip:udp:data',
 'eth:ethertype:ip:icmp:ip:tcp',
 'eth:ethertype:ip:icmp:ip:udp',
 'eth:ethertype:ip:icmp:ip:udp:coap',
 'eth:ethertype:ip:icmp:ip:udp:data',
 'eth:ethertype:ip:icmp:ip:udp:dns',
 'eth:ethertype:ip:icmp:ip:udp:gtp',
 'eth:ethertype:ip:icmp:ip:udp:kerberos',
 'eth:ethertype:ip:icmp:ip:udp:l2tp',
 'eth:ethertype:ip:icmp:ip:udp:memcache',
 'eth:ethertype:ip:icmp:ip:udp:nbns',
 'eth:ethertype:ip:icmp:ip:udp:ntp',
 'eth:ethertype:ip:icmp:ip:udp:openvpn',
 'eth:ethertype:ip:icmp:ip:udp:portcontrol',
 'eth:ethertype:ip:icmp:ip:udp:rip',
 'eth:ethertype:ip:icmp:ip:udp:rpc',
 'eth:ethertype:ip:icmp:ip:udp:sip',
 'eth:ethertype:ip:icmp:ip:udp:snmp',
 'eth:ethertype:ip:icmp:ip:udp:ssdp',
 'eth:ethertype:ip:tcp',
 'eth:ethertype:ip:tcp:data',
 'eth:ethertype:ip:tcp:ftp',
 'eth:ethertype:ip:tcp:http',
 'eth:ethertype:ip:tcp:http:data',
 'eth:ethertype:ip:tcp:http:data-text-lines',
 'eth:ethertype:ip:tc

In [10]:
protocols_li[0][0]

'eth:ethertype:ip:tcp:http:data'

In [24]:
kf['frame.len'] = kf['frame.len'][0]
kf.head(10)

TypeError: Column assignment doesn't support type ndarray

In [34]:
# spark.catalog.refreshTable()
try_path = "/home/antslab/data_hdd4t/pcap_process/2020_01_10/中華電信/snort.2020-01-10_dir/snort.log/1578663586.json"
kf = ks.read_json(try_path)
kf.head(10)

,_index,_score,_source,_type
0,packets-2020-04-12,None,"((None, United States, googlebot.com, [7156], ...",pcap_file
1,packets-2020-04-12,None,"((None, United States, googlebot.com, [7156], ...",pcap_file
2,packets-2020-04-12,None,"((None, United States, googlebot.com, [7156], ...",pcap_file
3,packets-2020-04-12,None,"((None, United States, googlebot.com, [4320], ...",pcap_file
4,packets-2020-04-12,None,"((None, United States, googlebot.com, [66], [5...",pcap_file
5,packets-2020-04-12,None,"((None, United States, googlebot.com, [2902], ...",pcap_file
6,packets-2020-04-12,None,"((None, United States, googlebot.com, [66], [7...",pcap_file
7,packets-2020-04-12,None,"((None, United States, googlebot.com, [66], [8...",pcap_file
8,packets-2020-04-12,None,"((None, United States, googlebot.com, [66], [9...",pcap_file
9,packets-2020-04-12,None,"((None, United States, googlebot.com, [66], [1...",pcap_file


In [43]:
ss = kf.iloc[11,2]
ss

Row(layers=Row(city=None, country='United States', domain='googlebot.com', frame.len=['66'], frame.number=['12'], frame.protocols=['eth:ethertype:ip:tcp'], frame.time=['Jan 10, 2020 21:39:46.202354000 CST'], frame.time_epoch=['1578663586.202354000'], ip.dst=['210.242.145.157'], ip.proto=['6'], ip.src=['66.249.73.26'], ip.ttl=['103'], isp='Googlebot', latitude=37.751, longitude=-97.822, network='66.249.72.0/23', subdivisions=None, tcp.dstport=['80'], tcp.flags.ack=['1'], tcp.flags.fin=['0'], tcp.flags.push=['0'], tcp.flags.reset=['0'], tcp.flags.syn=['0'], tcp.flags.urg=['0'], tcp.srcport=['35750'], tcp.window_size_value=['7555'], traits_network='66.249.72.0/22'))

In [42]:
kf.iloc[10,2]

Row(layers=Row(city=None, country='United States', domain='googlebot.com', frame.len=['66'], frame.number=['11'], frame.protocols=['eth:ethertype:ip:tcp'], frame.time=['Jan 10, 2020 21:39:46.202300000 CST'], frame.time_epoch=['1578663586.202300000'], ip.dst=['210.242.145.136'], ip.proto=['6'], ip.src=['66.249.79.217'], ip.ttl=['104'], isp='Googlebot', latitude=37.751, longitude=-97.822, network='66.249.79.0/24', subdivisions=None, tcp.dstport=['80'], tcp.flags.ack=['1'], tcp.flags.fin=['0'], tcp.flags.push=['0'], tcp.flags.reset=['0'], tcp.flags.syn=['0'], tcp.flags.urg=['0'], tcp.srcport=['63070'], tcp.window_size_value=['5446'], traits_network='66.249.79.128/25'))

In [47]:
kf.iloc[0,2]

Row(layers=Row(city=None, country='United States', domain='googlebot.com', frame.len=['7156'], frame.number=['1'], frame.protocols=['eth:ethertype:ip:tcp:http:data'], frame.time=['Jan 10, 2020 21:39:46.201346000 CST'], frame.time_epoch=['1578663586.201346000'], ip.dst=['66.249.73.45'], ip.proto=['6'], ip.src=['210.242.149.151'], ip.ttl=['64'], isp='Googlebot', latitude=37.751, longitude=-97.822, network='66.249.72.0/23', subdivisions=None, tcp.dstport=['42630'], tcp.flags.ack=['1'], tcp.flags.fin=['0'], tcp.flags.push=['0'], tcp.flags.reset=['0'], tcp.flags.syn=['0'], tcp.flags.urg=['0'], tcp.srcport=['80'], tcp.window_size_value=['235'], traits_network='66.249.72.0/22'))

In [46]:
ss[0][5][0].split(':')[3]

['eth:ethertype:ip:tcp']

In [ ]:
try_path = "/home/antslab/data_hdd4t/pcap_process/2020_01_10/中華電信/snort.2020-01-10_dir/snort.log/1578663586.json"
kf = ks.read_json(try_path)

In [19]:
from pyspark.sql.functions import pandas_udf, PandasUDFType, udf
# @pandas_udf('long', PandasUDFType.SCALAR)
def get_protocol(s:pd.Series):
    try:
        protocol = s[0][5]
        protocol = protocol[0].split(':')[3]
    except KeyError:
        protocol = 'UDP'
    return protocol #print

In [4]:
try_path = "/home/antslab/data_hdd4t/pcap_process/2020_01_10/中華電信/snort.2020-01-10_dir/snort.log/1578663586.json"
kf = ks.read_json(try_path)
kf.iloc[0,2]

/opt/spark/python/pyspark/sql/dataframe.py:2184: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Nested StructType not supported in conversion to Arrow
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


Row(layers=Row(city=None, country='United States', domain='googlebot.com', frame.len=['7156'], frame.number=['1'], frame.protocols=['eth:ethertype:ip:tcp:http:data'], frame.time=['Jan 10, 2020 21:39:46.201346000 CST'], frame.time_epoch=['1578663586.201346000'], ip.dst=['66.249.73.45'], ip.proto=['6'], ip.src=['210.242.149.151'], ip.ttl=['64'], isp='Googlebot', latitude=37.751, longitude=-97.822, network='66.249.72.0/23', subdivisions=None, tcp.dstport=['42630'], tcp.flags.ack=['1'], tcp.flags.fin=['0'], tcp.flags.push=['0'], tcp.flags.reset=['0'], tcp.flags.syn=['0'], tcp.flags.urg=['0'], tcp.srcport=['80'], tcp.window_size_value=['235'], traits_network='66.249.72.0/22'))

In [12]:
try_path = "/home/antslab/data_hdd4t/pcap_process/2020_01_10/中華電信/snort.2020-01-10_dir/snort.log/1578663586.json"
sdf = spark.read.option("multiline","true").json(try_path)
sdf = sdf.select("_source.layers.*")
sdf.show()
# kf =sdf.to_koalas()

+------+-------------+-------------+---------+------------+--------------------+--------------------+--------------------+-----------------+--------+-----------------+------+---------+--------+---------+--------------+------------+-----------+-------------+-------------+--------------+---------------+-------------+-------------+-----------+---------------------+----------------+
|  city|      country|       domain|frame.len|frame.number|     frame.protocols|          frame.time|    frame.time_epoch|           ip.dst|ip.proto|           ip.src|ip.ttl|      isp|latitude|longitude|       network|subdivisions|tcp.dstport|tcp.flags.ack|tcp.flags.fin|tcp.flags.push|tcp.flags.reset|tcp.flags.syn|tcp.flags.urg|tcp.srcport|tcp.window_size_value|  traits_network|
+------+-------------+-------------+---------+------------+--------------------+--------------------+--------------------+-----------------+--------+-----------------+------+---------+--------+---------+--------------+------------+-----

In [13]:
kf = sdf.to_koalas()
kf.head(10)

,city,country,domain,frame.len,frame.number,frame.protocols,frame.time,frame.time_epoch,ip.dst,ip.proto,ip.src,ip.ttl,isp,latitude,longitude,network,subdivisions,tcp.dstport,tcp.flags.ack,tcp.flags.fin,tcp.flags.push,tcp.flags.reset,tcp.flags.syn,tcp.flags.urg,tcp.srcport,tcp.window_size_value,traits_network
0,None,United States,googlebot.com,[7156],[1],[eth:ethertype:ip:tcp:http:data],"[Jan 10, 2020 21:39:46.201346000 CST]",[1578663586.201346000],[66.249.73.45],[6],[210.242.149.151],[64],Googlebot,37.751,-97.822,66.249.72.0/23,None,[42630],[1],[0],[0],[0],[0],[0],[80],[235],66.249.72.0/22
1,None,United States,googlebot.com,[7156],[2],[eth:ethertype:ip:tcp:http:data],"[Jan 10, 2020 21:39:46.201398000 CST]",[1578663586.201398000],[66.249.73.45],[6],[210.242.149.151],[64],Googlebot,37.751,-97.822,66.249.72.0/23,None,[42630],[1],[0],[0],[0],[0],[0],[80],[235],66.249.72.0/22
2,None,United States,googlebot.com,[7156],[3],[eth:ethertype:ip:tcp:http:data],"[Jan 10, 2020 21:39:46.201456000 CST]",[1578663586.201456000],[66.249.73.45],[6],[210.242.149.151],[64],Googlebot,37.751,-97.822,66.249.72.0/23,None,[42630],[1],[0],[0],[0],[0],[0],[80],[235],66.249.72.0/22
3,None,United States,googlebot.com,[4320],[4],[eth:ethertype:ip:tcp:http:data],"[Jan 10, 2020 21:39:46.201509000 CST]",[1578663586.201509000],[66.249.73.45],[6],[210.242.149.151],[64],Googlebot,37.751,-97.822,66.249.72.0/23,None,[42630],[1],[0],[0],[0],[0],[0],[80],[235],66.249.72.0/22
4,None,United States,googlebot.com,[66],[5],[eth:ethertype:ip:tcp],"[Jan 10, 2020 21:39:46.201514000 CST]",[1578663586.201514000],[210.242.145.132],[6],[66.249.79.231],[104],Googlebot,37.751,-97.822,66.249.79.0/24,None,[80],[1],[0],[0],[0],[0],[0],[39498],[5280],66.249.79.128/25
5,None,United States,googlebot.com,[2902],[6],[eth:ethertype:ip:tcp:http:data],"[Jan 10, 2020 21:39:46.201742000 CST]",[1578663586.201742000],[66.249.79.231],[6],[210.242.145.132],[64],Googlebot,37.751,-97.822,66.249.79.0/24,None,[39498],[1],[0],[0],[0],[0],[0],[80],[235],66.249.79.128/25
6,None,United States,googlebot.com,[66],[7],[eth:ethertype:ip:tcp],"[Jan 10, 2020 21:39:46.202162000 CST]",[1578663586.202162000],[210.242.146.156],[6],[66.249.64.52],[104],Googlebot,37.751,-97.822,66.249.64.0/23,None,[80],[1],[0],[0],[0],[0],[0],[46948],[5632],66.249.64.0/22
7,None,United States,googlebot.com,[66],[8],[eth:ethertype:ip:tcp],"[Jan 10, 2020 21:39:46.202234000 CST]",[1578663586.202234000],[210.242.146.156],[6],[66.249.64.52],[104],Googlebot,37.751,-97.822,66.249.64.0/23,None,[80],[1],[0],[0],[0],[0],[0],[46948],[5636],66.249.64.0/22
8,None,United States,googlebot.com,[66],[9],[eth:ethertype:ip:tcp],"[Jan 10, 2020 21:39:46.202239000 CST]",[1578663586.202239000],[210.242.145.136],[6],[66.249.79.217],[104],Googlebot,37.751,-97.822,66.249.79.0/24,None,[80],[1],[0],[0],[0],[0],[0],[63070],[5450],66.249.79.128/25
9,None,United States,googlebot.com,[66],[10],[eth:ethertype:ip:tcp],"[Jan 10, 2020 21:39:46.202296000 CST]",[1578663586.202296000],[210.242.145.136],[6],[66.249.79.217],[104],Googlebot,37.751,-97.822,66.249.79.0/24,None,[80],[1],[0],[0],[0],[0],[0],[63070],[5438],66.249.79.128/25


In [11]:
from databricks.koalas import json_normalize

ImportError: cannot import name 'json_normalize' from 'databricks.koalas' (/home/antslab/anaconda3/lib/python3.7/site-packages/databricks/koalas/__init__.py)

In [8]:
kf.iloc[0,0]

{'city': None,
 'country': 'United States',
 'domain': 'googlebot.com',
 'frame.len': array(['7156'], dtype=object),
 'frame.number': array(['1'], dtype=object),
 'frame.protocols': array(['eth:ethertype:ip:tcp:http:data'], dtype=object),
 'frame.time': array(['Jan 10, 2020 21:39:46.201346000 CST'], dtype=object),
 'frame.time_epoch': array(['1578663586.201346000'], dtype=object),
 'ip.dst': array(['66.249.73.45'], dtype=object),
 'ip.proto': array(['6'], dtype=object),
 'ip.src': array(['210.242.149.151'], dtype=object),
 'ip.ttl': array(['64'], dtype=object),
 'isp': 'Googlebot',
 'latitude': 37.751,
 'longitude': -97.822,
 'network': '66.249.72.0/23',
 'subdivisions': None,
 'tcp.dstport': array(['42630'], dtype=object),
 'tcp.flags.ack': array(['1'], dtype=object),
 'tcp.flags.fin': array(['0'], dtype=object),
 'tcp.flags.push': array(['0'], dtype=object),
 'tcp.flags.reset': array(['0'], dtype=object),
 'tcp.flags.syn': array(['0'], dtype=object),
 'tcp.flags.urg': array(['0'], dt

In [14]:
# kf =sdf.to_koalas()
def test(s):
    return 'UDP_TCP'
#     except:
#         return 'TCP' + str(s[0])
# kf['protocol'] = None
kf['protocol'] = kf['city'].map(test)
# kf['_source'] = kf._source.apply(test)
kf.head(10)

,city,country,domain,frame.len,frame.number,frame.protocols,frame.time,frame.time_epoch,ip.dst,ip.proto,ip.src,ip.ttl,isp,latitude,longitude,network,subdivisions,tcp.dstport,tcp.flags.ack,tcp.flags.fin,tcp.flags.push,tcp.flags.reset,tcp.flags.syn,tcp.flags.urg,tcp.srcport,tcp.window_size_value,traits_network,protocol
0,None,United States,googlebot.com,[7156],[1],[eth:ethertype:ip:tcp:http:data],"[Jan 10, 2020 21:39:46.201346000 CST]",[1578663586.201346000],[66.249.73.45],[6],[210.242.149.151],[64],Googlebot,37.751,-97.822,66.249.72.0/23,None,[42630],[1],[0],[0],[0],[0],[0],[80],[235],66.249.72.0/22,UDP_TCP
1,None,United States,googlebot.com,[7156],[2],[eth:ethertype:ip:tcp:http:data],"[Jan 10, 2020 21:39:46.201398000 CST]",[1578663586.201398000],[66.249.73.45],[6],[210.242.149.151],[64],Googlebot,37.751,-97.822,66.249.72.0/23,None,[42630],[1],[0],[0],[0],[0],[0],[80],[235],66.249.72.0/22,UDP_TCP
2,None,United States,googlebot.com,[7156],[3],[eth:ethertype:ip:tcp:http:data],"[Jan 10, 2020 21:39:46.201456000 CST]",[1578663586.201456000],[66.249.73.45],[6],[210.242.149.151],[64],Googlebot,37.751,-97.822,66.249.72.0/23,None,[42630],[1],[0],[0],[0],[0],[0],[80],[235],66.249.72.0/22,UDP_TCP
3,None,United States,googlebot.com,[4320],[4],[eth:ethertype:ip:tcp:http:data],"[Jan 10, 2020 21:39:46.201509000 CST]",[1578663586.201509000],[66.249.73.45],[6],[210.242.149.151],[64],Googlebot,37.751,-97.822,66.249.72.0/23,None,[42630],[1],[0],[0],[0],[0],[0],[80],[235],66.249.72.0/22,UDP_TCP
4,None,United States,googlebot.com,[66],[5],[eth:ethertype:ip:tcp],"[Jan 10, 2020 21:39:46.201514000 CST]",[1578663586.201514000],[210.242.145.132],[6],[66.249.79.231],[104],Googlebot,37.751,-97.822,66.249.79.0/24,None,[80],[1],[0],[0],[0],[0],[0],[39498],[5280],66.249.79.128/25,UDP_TCP
5,None,United States,googlebot.com,[2902],[6],[eth:ethertype:ip:tcp:http:data],"[Jan 10, 2020 21:39:46.201742000 CST]",[1578663586.201742000],[66.249.79.231],[6],[210.242.145.132],[64],Googlebot,37.751,-97.822,66.249.79.0/24,None,[39498],[1],[0],[0],[0],[0],[0],[80],[235],66.249.79.128/25,UDP_TCP
6,None,United States,googlebot.com,[66],[7],[eth:ethertype:ip:tcp],"[Jan 10, 2020 21:39:46.202162000 CST]",[1578663586.202162000],[210.242.146.156],[6],[66.249.64.52],[104],Googlebot,37.751,-97.822,66.249.64.0/23,None,[80],[1],[0],[0],[0],[0],[0],[46948],[5632],66.249.64.0/22,UDP_TCP
7,None,United States,googlebot.com,[66],[8],[eth:ethertype:ip:tcp],"[Jan 10, 2020 21:39:46.202234000 CST]",[1578663586.202234000],[210.242.146.156],[6],[66.249.64.52],[104],Googlebot,37.751,-97.822,66.249.64.0/23,None,[80],[1],[0],[0],[0],[0],[0],[46948],[5636],66.249.64.0/22,UDP_TCP
8,None,United States,googlebot.com,[66],[9],[eth:ethertype:ip:tcp],"[Jan 10, 2020 21:39:46.202239000 CST]",[1578663586.202239000],[210.242.145.136],[6],[66.249.79.217],[104],Googlebot,37.751,-97.822,66.249.79.0/24,None,[80],[1],[0],[0],[0],[0],[0],[63070],[5450],66.249.79.128/25,UDP_TCP
9,None,United States,googlebot.com,[66],[10],[eth:ethertype:ip:tcp],"[Jan 10, 2020 21:39:46.202296000 CST]",[1578663586.202296000],[210.242.145.136],[6],[66.249.79.217],[104],Googlebot,37.751,-97.822,66.249.79.0/24,None,[80],[1],[0],[0],[0],[0],[0],[63070],[5438],66.249.79.128/25,UDP_TCP


In [11]:
ss = kf.iloc[0,0]
ss

/opt/spark/python/pyspark/sql/dataframe.py:2222: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true, but has reached the error below and can not continue. Note that 'spark.sql.execution.arrow.pyspark.fallback.enabled' does not have an effect on failures in the middle of computation.
  An error occurred while calling o1886.getResult.
: org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:227)
	at org.apache.spark.security.SocketAuthServer.getResult(SocketAuthServer.scala:88)
	at org.apache.spark.security.SocketAuthServer.getResult(SocketAuthServer.scala:84)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498

Py4JJavaError: An error occurred while calling o1886.getResult.
: org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:227)
	at org.apache.spark.security.SocketAuthServer.getResult(SocketAuthServer.scala:88)
	at org.apache.spark.security.SocketAuthServer.getResult(SocketAuthServer.scala:84)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection

In [5]:
# kf =sdf.to_koalas()
kf.head(10)
# sdf.show()

/opt/spark/python/pyspark/sql/dataframe.py:2184: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Nested StructType not supported in conversion to Arrow
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


,_index,_score,_source,_type
0,packets-2020-04-12,None,"((None, United States, googlebot.com, [7156], ...",pcap_file
1,packets-2020-04-12,None,"((None, United States, googlebot.com, [7156], ...",pcap_file
2,packets-2020-04-12,None,"((None, United States, googlebot.com, [7156], ...",pcap_file
3,packets-2020-04-12,None,"((None, United States, googlebot.com, [4320], ...",pcap_file
4,packets-2020-04-12,None,"((None, United States, googlebot.com, [66], [5...",pcap_file
5,packets-2020-04-12,None,"((None, United States, googlebot.com, [2902], ...",pcap_file
6,packets-2020-04-12,None,"((None, United States, googlebot.com, [66], [7...",pcap_file
7,packets-2020-04-12,None,"((None, United States, googlebot.com, [66], [8...",pcap_file
8,packets-2020-04-12,None,"((None, United States, googlebot.com, [66], [9...",pcap_file
9,packets-2020-04-12,None,"((None, United States, googlebot.com, [66], [1...",pcap_file


In [18]:
import pyarrow
pyarrow.__version__

'0.16.0'

In [20]:
# kdf = kf[0:9999]
kf['protocol'] = kf._source.map(get_protocol)
kf.head(10)

Py4JJavaError: An error occurred while calling o2052.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 32.0 failed 1 times, most recent failure: Lost task 0.0 in stage 32.0 (TID 267, 192.168.50.123, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 597, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 589, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 398, in dump_stream
    return ArrowStreamSerializer.dump_stream(self, init_stream_yield_batches(), stream)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 234, in dump_stream
    for batch in iterator:
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 391, in init_stream_yield_batches
    for series in itera

Py4JJavaError: An error occurred while calling o2171.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 34.0 failed 1 times, most recent failure: Lost task 0.0 in stage 34.0 (TID 271, 192.168.50.123, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 597, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 589, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 398, in dump_stream
    return ArrowStreamSerializer.dump_stream(self, init_stream_yield_batches(), stream)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 234, in dump_stream
    for batch in iterator:
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 391, in init_stream_yield_batches
    for series in itera

一樣的session
* 同city=None, country='United States'、frame.len、frame.protocols、ip.proto、ip.src、ip.ttl、isp、latitude、longitude、network、tcp.dstport、tcp.srcport、traits_network

In [25]:
kf#.head(10)

Py4JJavaError: An error occurred while calling o2793.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 50.0 failed 1 times, most recent failure: Lost task 0.0 in stage 50.0 (TID 299, 192.168.50.123, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 597, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 589, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 398, in dump_stream
    return ArrowStreamSerializer.dump_stream(self, init_stream_yield_batches(), stream)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 234, in dump_stream
    for batch in iterator:
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 391, in init_stream_yield_batches
    for series in itera

Py4JJavaError: An error occurred while calling o2911.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 52.0 failed 1 times, most recent failure: Lost task 0.0 in stage 52.0 (TID 303, 192.168.50.123, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 597, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 589, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 398, in dump_stream
    return ArrowStreamSerializer.dump_stream(self, init_stream_yield_batches(), stream)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 234, in dump_stream
    for batch in iterator:
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 391, in init_stream_yield_batches
    for series in itera

In [44]:
import cupy as cp

In [10]:
import cupy as cp
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm

In [29]:
proto_li = ['http','smb','telnet','ftp','smtp','mysql','ssh','rpc','imap','pop','sip','tds'] #改!!'tds',
date_li2 = ["0106","0107","0108","0109","0110","0111","0112"] 
date_li = ["20200106","20200107","20200108","20200109","20200110","20200111","20200112"]  #改!!
# date_li = ['0102','0130','0110']
file_name = "_".join(sorted(date_li2)) #改!!
min_date = str(min(date_li))
for i,proto in enumerate(proto_li):
    if i==0:
        all_df = pickle.load(file=open('/home/antslab/spark_data/pcap_inter/'+str(min_date[:4])+'_'+str(min_date[4:6])+'_'+str(min_date[6:])+'/中華電信/case_pickles/'+proto+'_trend_df_'+file_name+'.pkl','rb')) #改!!)
        all_df['idx'] = all_df['idx']+'_'+proto
    else:
        temp = pickle.load(file=open('/home/antslab/spark_data/pcap_inter/'+str(min_date[:4])+'_'+str(min_date[4:6])+'_'+str(min_date[6:])+'/中華電信/case_pickles/'+proto+'_trend_df_'+file_name+'.pkl','rb')) #改!!
        temp['idx'] = temp['idx']+'_'+proto
        all_df = all_df.append(temp)
all_df = all_df.reset_index(drop=True)
all_df

,idx,timestamp,country,src_ip,ssdeep
0,0106_213_http,"[1578239432.052566, 1578239434.409594, 1578239...","[Hong Kong, China, China, Hong Kong, Hong Kong...","[119.28.222.106, 121.201.46.229, 61.160.223.17...",[12:7YjpXcHnMQ7s95HfCijmHXq/I75cuXNFljhSR5ZDxm...
1,0106_252_http,"[1578239437.787893, 1578239440.191034, 1578239...","[United States, Brazil, Brazil, Russia, Brazil...","[216.244.66.199, 200.153.231.153, 200.153.231....",[12:rHXq6xmMRefGI/XES+AoGY1uqh5U9Z1J3vhm:Da7M2...
2,0106_3766_http,"[1578239712.963036, 1578239721.816867, 1578240...","[Brazil, Morocco, Taiwan, Iraq, United States,...","[45.6.168.140, 41.251.180.47, 220.229.71.153, ...",[12:rHXAxmMRejCgIs0+R56Y1awh5UPlVFYNjZpvhm:DFM...
3,0107_174_http,"[1578325830.146078, 1578325832.7705, 157832583...","[China, China, China, Russia, Canada, China, U...","[122.114.239.196, 122.114.239.196, 122.114.239...",96:DBPrJk3m5wzznXFm5wzEm5wzn3UaHrnXFUa2pUa4m3E...
4,0110_124_http,"[1578585029.371207, 1578585029.595845, 1578585...","[Russia, Russia, Russia, Russia, Russia, Russi...","[5.188.211.35, 5.188.211.35, 5.188.211.35, 5.1...",96:FZpnXFLZLhZc34kYOFnXF4kYO+4kYOh3UOQnXFUOxUO...
...,...,...,...,...,...
751,0106_94489308925_tds,"[1578244005.404201, 1578249937.289241, 1578278...","[Taiwan, India, Vietnam, Taiwan, Taiwan, Taiwa...","[210.242.49.2, 210.212.235.51, 171.243.196.82,...",[12:Mlvl9+VTv6LrOIDuTclwBk6LA+Oi5smXv3:MwxEOID...
752,0107_8589939281_tds,"[1578382169.519574, 1578382258.060414, 1578382...","[Poland, Poland, Poland, Poland, Poland, Polan...","[185.234.219.85, 185.234.219.85, 185.234.219.8...",[12:ndWl2qsH9ssssFPwhcQPyEHfdWl2qRdxweIfksV0:n...
753,0106_34359740430_tds,"[1578248899.847321, 1578292404.613339, 1578300...","[India, India, Philippines, Philippines, Vietn...","[210.212.235.51, 210.212.202.99, 115.85.46.242...",[12:Mh9qX9C8mKTvZ8kxIXwVUw6A+Oi5sovv3:Mh6aMbxi...
754,0106_3739_tds,"[1578239974.827253, 1578241552.063305, 1578242...","[China, China, China, China, China, China]","[152.136.89.155, 152.136.89.155, 152.136.89.15...",6:O9F1D9sC9CwH/I/GD8qeM4E+mao2W+W168VpWm75YV:g...


In [41]:
weird_ips_set = set(weird_ips)
def if_weird(s_ip):
    s_ip_set = set(s_ip)
    temp = weird_ips_set.intersection(s_ip_set)
    if len(temp)>0:
        return True
    else:
        return False
all_df['weird'] = all_df.src_ip.apply(if_weird)
all_df[all_df['weird']==True]

,idx,timestamp,country,src_ip,ssdeep,weird
722,0106_38_tds,"[1578239428.513828, 1578239428.527958, 1578239...","[Philippines, Russia, Japan, Japan, Russia, Ja...","[210.213.73.170, 93.81.185.139, 1.33.83.132, 1...",[12:MdjfmxTvuw36ItVE6YlBht4xaUpA+Oi5O/nJ:M49uw...,True
723,0106_13_tds,"[1578239428.699379, 1578239429.100849, 1578239...","[Japan, Russia, Russia, Russia, Russia, Philip...","[1.33.83.132, 93.81.185.139, 93.81.185.139, 93...",[3:OfVGX4V9VfV1V1V1V1UQ91BVWfg0VZ39LfVbHfVYN1V...,True
724,0106_289_tds,"[1578239429.447062, 1578239429.474265, 1578239...","[Turkey, Brazil, Turkey, Turkey, Philippines, ...","[217.131.4.46, 177.38.54.186, 217.131.4.46, 21...",[12:M5epX+xTvuw3zLw9VCUn/6WfEFO+vXVyAA+Oi5OOVl...,True
725,0106_257698049845_tds,"[1578247196.969601, 1578256397.518171, 1578334...","[Taiwan, Taiwan, China, China, China, China, C...","[210.242.49.2, 210.242.49.2, 210.36.22.214, 21...",[12:MfTxTvuw3kG6OGB3eVxVZ+ytZpP9YVoRA+Oi5spJ:M...,True
726,0106_2264_tds,"[1578239446.030398, 1578239450.34886, 15782394...","[Taiwan, Brazil, Taiwan, Taiwan, Taiwan, Russi...","[210.59.176.37, 177.38.54.186, 210.242.49.2, 2...",[12:MtSyxTvuw3d4AJG6Ai5KOm1NUGheU2A+Oi5OBVlJ:M...,True
727,0106_5969_tds,"[1578239431.361256, 1578239446.145513, 1578239...","[Philippines, Russia, Taiwan, Philippines, Tai...","[210.213.73.170, 93.81.185.139, 210.242.49.2, ...",[6:My/fBd+T9uTv6LCNMWbQcTov/5Ky9TXTtAvW9cfKDoc...,True
728,0106_755_tds,"[1578239431.817301, 1578239431.849344, 1578239...","[Philippines, Philippines, Russia, Taiwan, Rus...","[210.213.73.170, 210.213.73.170, 93.81.185.139...",[12:MLlsKTvctV9rV94ly5Y9ThgsssssssKhA+Oi5acovv...,True
729,0106_312_tds,"[1578239429.470633, 1578239431.677342, 1578239...","[Brazil, South Korea, Taiwan, Taiwan, Philippi...","[177.38.54.186, 115.161.187.129, 210.59.176.37...",[12:MA1epX+nMXTv6LFSXvsX8IXOdS/if4/htXshA+Oi5s...,True
730,0108_4060_tds,"[1578412247.14168, 1578412557.370859, 15784125...","[Taiwan, South Korea, China, Ukraine, Egypt, C...","[210.242.156.9, 210.109.97.141, 210.36.22.214,...","[3:/:/, 3:/:/]",True
731,0106_8874_tds,"[1578239505.553952, 1578240088.276197, 1578240...","[Taiwan, Turkey, Turkey, Taiwan, Taiwan, Taiwa...","[210.242.49.2, 217.131.4.46, 217.131.4.46, 210...",[12:MdOTFAxTvuw3xNdtD+f5oVUVQA+Oi5sEvXJ:MQTFA9...,True


In [42]:
#find all ips
all_ips = all_df.src_ip.tolist()
all_ips = sum(all_ips,[])
all_ips = list(set(all_ips))
all_ips = sorted(all_ips)
# col_li = ['Country']
# col_li.extend(all_df.idx.tolist())
col_li = all_df.idx.tolist()
col_li = [x for x in col_li if 'tds' not in x]
col_li.append('tds') #改!!
jc_matrix = pd.DataFrame(0, index=all_ips, columns=col_li)
col_li

['0106_213_http',
 '0106_252_http',
 '0106_3766_http',
 '0107_174_http',
 '0110_124_http',
 '0109_215_http',
 '0106_417_http',
 '0106_519_http',
 '0106_3_http',
 '0106_14398_http',
 '0106_15438_http',
 '0106_8935_http',
 '0106_3026_http',
 '0107_17179906661_http',
 '0112_105_http',
 '0106_2792_http',
 '0106_4752_http',
 '0107_128849034994_http',
 '0106_14909_http',
 '0106_8319_http',
 '0106_77309431474_http',
 '0106_4080_http',
 '0107_68719497531_http',
 '0106_32565_http',
 '0106_15392_http',
 '0107_35099_http',
 '0106_550_http',
 '0106_2338_http',
 '0106_25769836623_http',
 '0109_307_http',
 '0109_50_http',
 '0106_467_http',
 '0106_4763_http',
 '0106_8589954022_http',
 '0106_201_http',
 '0106_2351_http',
 '0107_16005_http',
 '0106_51539638239_http',
 '0107_4581_http',
 '0106_8589954986_http',
 '0106_8589956685_http',
 '0106_15454_http',
 '0106_42949688737_http',
 '0109_411_http',
 '0106_8309_http',
 '0106_31759_http',
 '0106_19197_http',
 '0106_25295_http',
 '0106_5146_http',
 '0108_1

In [25]:
select_df = all_df[all_df.idx.str.contains('tds')]
ip_li = list(select_df.src_ip.values)
len(ip_li)

34

In [28]:
col = '0106_77309431474_http'
select_df = all_df[all_df.idx == col]
ip_li = list(select_df.src_ip.values)
len(ip_li)

1

In [43]:
for col in tqdm(jc_matrix.columns.tolist()):
    if 'tds' in col: # 統一tds欄位，如果有的話
        select_df = all_df[all_df.idx.str.contains('tds')]
        ip_li = list(select_df.src_ip.values)
        try:
            for ips in ip_li:
                jc_matrix.loc[ips,'tds'] = 1
#             jc_matrix = jc_matrix.drop([col])
        except IndexError:
            print("Didn't load tds protocol to all_ip. SKIPPING!")
            pass
    else:
        select_df = all_df[all_df.idx == col]
        ip_li = list(select_df.src_ip.values)
        for ips in ip_li:      
            jc_matrix.loc[ips,col] = 1
jc_matrix['np_array'] = list(jc_matrix.values)#.ravel()
jc_matrix

100%|██████████| 723/723 [00:08<00:00, 85.73it/s]


,0106_213_http,0106_252_http,0106_3766_http,0107_174_http,0110_124_http,0109_215_http,0106_417_http,0106_519_http,0106_3_http,0106_14398_http,...,0107_420906828667_pop,0108_111669167541_pop,0110_272406_pop,0112_309237649836_pop,0106_51539621821_sip,0106_465_sip,0106_15413_sip,0110_3451217_sip,tds,np_array
1.0.128.212,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1.0.134.96,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1.0.135.97,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1.0.139.246,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1.0.153.160,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99.81.0.86,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
99.89.116.178,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
99.89.92.55,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
99.93.68.47,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [44]:
def sum_val(npy):
    return npy.sum()
jc_matrix['sum'] = jc_matrix.np_array.apply(sum_val)
temp = jc_matrix[jc_matrix['sum'] == 0]
weird_ips = temp.index.tolist()
weird_ips

[]

In [40]:
jc_matrix[jc_matrix.index == '1.71.189.100']

,0106_213_http,0106_252_http,0106_3766_http,0107_174_http,0110_124_http,0109_215_http,0106_417_http,0106_519_http,0106_3_http,0106_14398_http,...,0108_111669167541_pop,0110_272406_pop,0112_309237649836_pop,0106_51539621821_sip,0106_465_sip,0106_15413_sip,0110_3451217_sip,TDS,np_array,sum
1.71.189.100,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0


In [4]:
df = pickle.load(open('/home/antslab/spark_data/pcap_inter/2020_01_06/中華電信/case_pickles/'+
                      'CorrelateIP_APTIP_thr0.9_0106_0107_0108_0109_0110_0111_0112.pkl','rb'))
df

({'1.0.128.212': ['1.0.135.97',
   '1.0.139.246',
   '1.0.158.208',
   '1.0.166.195',
   '1.0.180.28',
   '1.0.186.181',
   '1.0.187.191',
   '1.0.194.218',
   '1.0.205.142',
   '1.0.205.50',
   '1.0.240.206',
   '1.0.243.72',
   '1.1.132.18',
   '1.1.133.23',
   '1.1.133.54',
   '1.1.133.66',
   '1.1.143.151',
   '1.1.148.7',
   '1.1.150.95',
   '1.1.156.160',
   '1.1.162.28',
   '1.1.164.161',
   '1.1.171.230',
   '1.1.181.209',
   '1.1.182.129',
   '1.1.183.76',
   '1.1.185.43',
   '1.1.187.116',
   '1.1.197.128',
   '1.1.197.163',
   '1.1.201.67',
   '1.1.205.91',
   '1.1.207.16',
   '1.1.208.143',
   '1.1.216.168',
   '1.1.216.32',
   '1.1.216.73',
   '1.1.218.137',
   '1.1.219.173',
   '1.1.219.197',
   '1.1.219.248',
   '1.1.226.176',
   '1.1.226.205',
   '1.1.230.225',
   '1.1.231.147',
   '1.1.233.215',
   '1.1.233.239',
   '1.1.234.19',
   '1.1.241.100',
   '1.1.242.199',
   '1.1.245.54',
   '1.1.247.185',
   '1.1.247.92',
   '1.1.255.135',
   '1.1.255.181',
   '1.10.132.97',

In [6]:
len(df[1])

4028